In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [11]:
import os
import glob
import torch
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count

# Load Data + YOLO

In [12]:
# load yolo
def load_model():
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    return model

model = load_model()

Using cache found in /home/landon/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-19 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [13]:
cwd = os.getcwd()
parent = os.path.dirname(cwd)
print(parent)
print(cwd)

/home/landon/Documents/Deep-Learning/DSAN6600Proj
/home/landon/Documents/Deep-Learning/DSAN6600Proj/code


In [14]:
#set parent as current working directory
cwd = parent
print(cwd)

/home/landon/Documents/Deep-Learning/DSAN6600Proj


In [15]:
project_path = cwd
raw_imgs_path = os.path.join(project_path, 'data/raw')
test_imgs_path = os.path.join(project_path, 'data/test-images')
all_imgs = glob.glob(os.path.join(raw_imgs_path, '*.jpg'))
test_imgs = glob.glob(os.path.join(test_imgs_path, '*.jpg'))

### TESTING ON SUBSET ###
#all_imgs = all_imgs[:200]
batch_size = int(len(all_imgs) * 0.07)

print(raw_imgs_path)
print(all_imgs[:3])

/home/landon/Documents/Deep-Learning/DSAN6600Proj/data/raw
['/home/landon/Documents/Deep-Learning/DSAN6600Proj/data/raw/style_romanticism_index_73633.jpg', '/home/landon/Documents/Deep-Learning/DSAN6600Proj/data/raw/style_impressionism_index_35454.jpg', '/home/landon/Documents/Deep-Learning/DSAN6600Proj/data/raw/style_impressionism_index_34401.jpg']


In [16]:
def process_images(model, img_paths):
    imgs = [Image.open(img_path).convert('RGB') for img_path in img_paths]
    results = model(imgs)
    dets = results.pandas().xyxy
    people_det = [det[(det['class'] == 0) & (det['confidence'] >= 0.7)] for det in dets]
    return people_det
def worker(img_paths):
    return process_images(model, img_paths)

In [17]:
print(os.path.join(project_path, 'data/subjectbox'))

/home/landon/Documents/Deep-Learning/DSAN6600Proj/data/subjectbox


In [18]:
def crop_and_save(img_paths, detections_list, verbose=False):
  out_path = os.path.join(project_path, 'data/subjectbox')
  os.makedirs(out_path, exist_ok=True)

  for img_path, detections in zip(img_paths, detections_list):
    img = Image.open(img_path)
    if detections.empty:
      if verbose:
        print(f"No detections for {img_path}")
      continue

    if verbose:
      n_crops = len(detections)
      plt.figure(figsize=(5 * max(1, n_crops), 6))
      plt.subplot(1, n_crops + 1, 1)
      plt.imshow(img)
      plt.title('Original Image')
      plt.axis('off')
    i = 2

    for index, row in detections.iterrows():
      xmin, ymin, xmax, ymax = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
      cropped_img = img.crop((xmin, ymin, xmax, ymax))
      img_name_no_ext = os.path.splitext(os.path.basename(img_path))[0]
      new_img_name = f"{img_name_no_ext}_subject_box_{xmin}_{ymin}_{xmax}_{ymax}.jpg"
      save_path = os.path.join(out_path, new_img_name)
      cropped_img.save(save_path)

      if verbose:
        plt.subplot(1, n_crops + 1, i)
        plt.imshow(cropped_img)
        plt.title(f'Cropped {i-1}')
        plt.axis('off')
        i += 1

In [19]:
import logging
from multiprocessing import Pool, set_start_method
import torch

# Ensure the start method is 'spawn'
set_start_method('spawn', force=True)

# Logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s - %(levelname)s')

# Assuming test_imgs is defined and worker, crop_and_save functions are implemented properly
batch_size = 500
batches = [test_imgs[i:i + batch_size] for i in range(0, len(test_imgs), batch_size)]
num_processes = 4

with Pool(processes=num_processes) as pool:
    for batch in batches:
        for img_path, result in zip(batch, pool.imap_unordered(worker, [batch])):
            if result:
                crop_and_save(img_path, result)
            else:
                print("No detections for this image:", img_path)


Process SpawnPoolWorker-10:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'worker' on <module '__main__' (built-in)>


KeyboardInterrupt: 